### Imports

In [1]:
pip install annoy

     |████████████████████████████████| 646 kB 4.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391619 sha256=86e327a51f641f1bdb4c28c4d1d61d5c404632ac9c9bb4937235f3e47efb10c1
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from fastai import *
from fastai.vision import *
from fastai.callbacks import *

np.random.seed(42)
torch.cuda.set_device(0)

from annoy import AnnoyIndex

import glob

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Creating DFs

In [5]:
#creating df of image names & dataset label (train, val, test)
img_partitions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Eval/list_eval_partition.csv', sep="\t", header=1, error_bad_lines=False, encoding="euc-kr")
img_partitions['path'] = img_partitions['path'].replace("K-Fashion 이미지/원천데이터/레트로/","")
img_partitions

,path,evaluation_status
0,K-Fashion 이미지/원천데이터/레트로/(121)IMG_1.jpg,val
1,K-Fashion 이미지/원천데이터/레트로/(211)IMG_1.jpg,val
2,"K-Fashion 이미지/원천데이터/레트로/018-285-4,257-35,865-H...",val
3,"K-Fashion 이미지/원천데이터/레트로/019-286-3,261-39,126-H...",train
4,K-Fashion 이미지/원천데이터/레트로/1-(9)-004-271-3031-516...,val
...,...,...
1176,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4876.jpg,train
1177,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4896.jpg,train
1178,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4916.jpg,val
1179,K-Fashion 이미지/원천데이터/레트로/팬츠-1-u_155677953310-00...,test


In [6]:
'''
#creating df of image names & dataset label (train, val, test)
img_partitions = pd.read_csv('./Eval/list_eval_partition.txt', delim_whitespace=True, header=1)

#creating df of attribute names and type label
attributes = pd.read_csv(
    './Anno/list_attr_cloth.txt', 
    sep='\s{2,}', 
    header=0, 
    skiprows=[0], 
    engine='python')

attributes_imgs = pd.read_csv(
    './Anno/list_attr_img.txt', 
    delim_whitespace=True, 
    header=0,
    skiprows=[0],
    names=(['image_name'] + attributes['attribute_name'].tolist()))
attributes_imgs.replace([-1,0,1],[False,False,True], inplace=True)

#creating df of image names, attribute tags, and dataset label
attributes_df = attributes_imgs.merge(img_partitions, how ='left', on='image_name')'''

#creating df of attribute names and type label
attributes = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Anno/list_attr_cloth.txt', 
    sep='\s{2,}', header=0, skiprows=[0], engine='python')
attributes.rename(columns={'material': 'shape'},inplace = True)
attributes

,shape,attribute_type
0,비즈,1
1,단추,1
2,니트꽈배기,1
3,체인,1
4,컷오프,1
5,더블브레스티드,1
6,드롭숄더,1
7,자수,1
8,프릴,1
9,프린지,1


In [7]:
attributes_imgs = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Anno/list_img.csv', 
   sep="\t", skiprows=[0],header=0, error_bad_lines=False, encoding="euc-kr",engine='python'
    )

# for data in attributes_imgs['material'] :
def img(x):
  if type(x) != str:
    return ""
  x = x.replace("['", "").replace("']", "").replace("', '", ",").split(",")
  return x[0]

attributes_imgs['material'] = attributes_imgs['material'].apply(lambda x: img(x))

attributes_imgs['print'] =  attributes_imgs['print'].apply(lambda x: img(x))

attributes_imgs['detail'] =  attributes_imgs['detail'].apply(lambda x: img(x))

attributes_imgs


,path,clothes,name,category,color,detail,print,material,length,neckline,collar,fit,shape
0,K-Fashion 이미지/원천데이터/레트로/(121)IMG_1.jpg,상의,(121)IMG_1.jpg,블라우스,네이비,,플로럴,우븐,긴팔,NaN,NaN,루즈,NaN
1,K-Fashion 이미지/원천데이터/레트로/(211)IMG_1.jpg,상의,(211)IMG_1.jpg,블라우스,베이지,페플럼,플로럴,시폰,긴팔,NaN,NaN,노멀,NaN
2,"K-Fashion 이미지/원천데이터/레트로/018-285-4,257-35,865-H...",원피스,"018-285-4,257-35,865-Hanna_409_07.jpg",드레스,화이트,퍼프,깅엄,우븐,반팔,스위트하트,NaN,루즈,NaN
3,"K-Fashion 이미지/원천데이터/레트로/019-286-3,261-39,126-H...",상의,"019-286-3,261-39,126-Haemil_194_06.jpg",니트웨어,와인,,무지,니트,7부소매,터틀넥,NaN,노멀,NaN
4,K-Fashion 이미지/원천데이터/레트로/1-(9)-004-271-3031-516...,상의,1-(9)-004-271-3031-5163-TOP_179_11.jpg,블라우스,화이트,,그래픽,우븐,7부소매,라운드넥,NaN,루즈,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4876.jpg,원피스,없음-994-994-완-DSC_4876.jpg,드레스,와인,플레어,플로럴,우븐,긴팔,NaN,NaN,노멀,NaN
1177,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4896.jpg,원피스,없음-994-994-완-DSC_4896.jpg,드레스,와인,플레어,플로럴,우븐,긴팔,NaN,NaN,노멀,NaN
1178,K-Fashion 이미지/원천데이터/레트로/없음-994-994-완-DSC_4916.jpg,원피스,없음-994-994-완-DSC_4916.jpg,드레스,와인,플레어,플로럴,우븐,긴팔,NaN,NaN,노멀,NaN
1179,K-Fashion 이미지/원천데이터/레트로/팬츠-1-u_155677953310-00...,상의,팬츠-1-u_155677953310-001 (1995).jpg,니트웨어,베이지,,아가일,니트,긴팔,라운드넥,NaN,노멀,NaN


In [8]:
#creating df of image names, attribute tags, and dataset label
#attributes_imgs.replace([-1,0,1],[False,False,True], inplace=True)
attributes_imgs = pd.merge(attributes_imgs, attributes, how='left', on='shape')
attributes_df = pd.merge(attributes_imgs, img_partitions, how='left', on='path')

attributes_df

ValueError: ignored

In [ ]:
#removing images from DF with no label
#training_df['max'] = [training_df.iloc[x][1:].max() for x in training_df.index.tolist()]
#training_df = training_df[training_df['max'].isin([True])].copy()
#training_df.drop('max', axis=1, inplace=True)
attributes_df['shape']=attributes_df['shape'].dropna()
attributes_df

In [ ]:
from fastai import *
from fastai.vision import *

np.random.seed(42)
torch.cuda.set_device(0)

In [ ]:
training_df = attributes_df[attributes_df['evaluation_status'].isin(['train','val'])][['shape','name']].copy()

#One Hot Encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

X = training_df.iloc[:, :].values
X= pd.DataFrame(X, columns=['shape','name'])
print(X)
'''
le = LabelEncoder()
print('****************************')
X[0] = le.fit_transform(X[0])
print(X)

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
print('****************************')
X = ct.fit_transform(X[0]).toarray()
print(X)
'''

pandas_one_hot_encoded = pd.get_dummies(X['shape'])

pandas_with_onehot = pd.concat(
    [DataFrame(X['name']),pandas_one_hot_encoded],axis=1)
pandas_with_onehot.replace([-1,0,1],[False,False,True], inplace=True)
print(pandas_with_onehot)


pandas_with_onehot.to_csv('/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Anno/OHEmaterial.csv', mode='w',encoding='utf-8-sig', header=True)
'''
csvfile = open('/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Anno/OHEmaterial.csv', 
   "w",newline="")

csvwriter = csv.writer(csvfile)
for rows in pandas_with_onehot :
  csvwriter.writerows(rows)

csvfile.close()
'''
'''
training_df = attributes_df[attributes_df['evaluation_status'].isin(['train','val'])][['name']].copy()
print(training_df)
print('****************************')
OHE_df = pd.concat(training_df, ohe)
OHE_df
'''
ohe = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/Anno/OHEmaterial.csv'
    )
print(ohe)
ohe=ohe.drop([ohe.columns[0],ohe.columns[2]],axis=1)
ohe

#attributes_df2 = ohe.merge(attributes_df, how ='left', on='name')
#attributes_df2

#training_df2 = attributes_df2[attributes_df2['evaluation_status'].isin(['train','val'])].copy()
#training_df2.reset_index(drop=True, inplace=True)
#training_df2 = training_df2.drop([training_df2.columns[19],training_df2.columns[20],training_df2.columns[21],training_df2.columns[22],training_df2.columns[23],training_df2.columns[24],training_df2.columns[25],training_df2.columns[26],training_df2.columns[27],training_df2.columns[28],training_df2.columns[29],training_df2.columns[30],training_df2.columns[31],training_df2.columns[32],training_df2.columns[33]],axis=1)
#training_df2

In [ ]:
def count_attr_imgs(attr_df):
    col_counts = {}

    for col in attr_df.columns[1:]:
        col_counts[col] = attr_df[col].value_counts().to_frame().loc[True][0]
    
    col_df = pd.DataFrame.from_dict(col_counts, orient='index', columns=['Count'])
    
    return col_df
#.sort_values(by='Count')

## Baseline Model (No Data Cleaning)

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(x=count_attr_imgs(training_df).sort_values(by='Count',ascending=False).index,y=count_attr_imgs(training_df).sort_values(by='Count',ascending=False)['Count'])
plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(x=count_attr_imgs(training_df).sort_values(by='Count',ascending=False)[:25].index,y=count_attr_imgs(training_df).sort_values(by='Count',ascending=False)[:25]['Count'])
plt.xticks(rotation=90);

In [ ]:
# take all the columns after the first "image_name" col
label_column_names = ohe.columns.tolist()[1:]
label_column_names


img_base_path = Path("/content/drive/MyDrive/Colab Notebooks/FashionRecommendations/CNN/K-Fashion 이미지/원천데이터/레트로/")
data = ImageDataBunch.from_df(img_base_path, ohe, ds_tfms=get_transforms(), size=150,label_col=label_column_names)
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=4, figsize=(14,12))

In [ ]:
print(data.classes)
len(data.classes),data.c


In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.1)
f_score = partial(fbeta, thresh=0.1)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[acc_02, f_score], callback_fns=ShowGraph)

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(4, slice(1e-5,1e-3))

In [ ]:
learn.save('shape-resnet50-size150-fr5-unfr4', return_path=True)
learn.export('shape-resnet50-size150-fr5-unfr4.pkl')

In [ ]:
predict_img_path = "floral_dress.jpg" 
category,classIndex,losses = learn.predict(open_image(predict_img_path))
predictions = sorted(zip(data.classes, map(float, losses)), key=lambda p: p[1], reverse=True)
print (predictions[:5])
show_image(open_image(predict_img_path))

# "/data/Michael/data/predict/2.png"